In [1]:
import yfinance as yf
import keras.backend as K
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler ,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score, mean_squared_error,mean_absolute_percentage_error,r2_score
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM,Dropout, BatchNormalization, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.models import Sequential
from keras.optimizer_v2.adam import Adam
from keras.optimizer_v2.rmsprop import RMSprop
import talib
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
# Get the data for the SPY ETF by specifying the stock ticker, start date, and end date
data = yf.download('0005.hk',"2017-09-28","2021-09-24")
# data.drop("Adj Close",axis=1,inplace=True)
data.tail(15)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-09-02,41.299999,41.549999,41.049999,41.349998,41.349998,20502999
2021-09-03,41.150002,41.799999,41.150002,41.450001,41.450001,29400411
2021-09-06,41.450001,41.700001,41.250000,41.400002,41.400002,24207987
2021-09-07,41.299999,41.400002,41.000000,41.000000,41.000000,23797121
2021-09-08,40.500000,40.849998,40.299999,40.450001,40.450001,21636170
2021-09-09,40.700001,41.000000,40.250000,40.400002,40.400002,16569193
2021-09-10,40.349998,40.950001,40.250000,40.900002,40.900002,7986921
2021-09-13,40.500000,40.849998,40.150002,40.700001,40.700001,8576584
2021-09-14,40.700001,41.099998,40.099998,40.200001,40.200001,19188693


In [3]:
df = data[['Close']]
#Create one more column Prediction shifted 15 days up. 
df['Prediction'] = data[['Close']].shift(-1)
#print data set
df=df.dropna()
print(df)

                Close  Prediction
Date                             
2017-09-28  76.199997   76.550003
2017-09-29  76.550003   77.449997
2017-10-03  77.449997   77.849998
2017-10-04  77.849998   77.349998
2017-10-06  77.349998   77.500000
...               ...         ...
2021-09-15  40.099998   40.000000
2021-09-16  40.000000   40.250000
2021-09-17  40.250000   39.000000
2021-09-20  39.000000   38.799999
2021-09-21  38.799999   39.750000

[982 rows x 2 columns]


/var/folders/8h/w0jn89bj2w5g85gj4ck7w0t00000gn/T/ipykernel_46622/1523880377.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Prediction'] = data[['Close']].shift(-1)


In [4]:
#Create a data set X and convert it into numpy array , which will be having actual values
X = np.array(df.drop(['Prediction'],1))
#Remove the last 15 rows
X = X
print(X)
# Create a dataset y which will be having Predicted values and convert into numpy array
y = np.array(df['Prediction'])
# Remove Last 15 rows
y = y
print(y)

[[76.19999695]
 [76.55000305]
 [77.44999695]
 [77.84999847]
 [77.34999847]
 [77.5       ]
 [78.59999847]
 [78.09999847]
 [77.30000305]
 [77.19999695]
 [77.30000305]
 [77.15000153]
 [77.        ]
 [76.        ]
 [76.5       ]
 [76.5       ]
 [76.5       ]
 [76.69999695]
 [76.90000153]
 [77.09999847]
 [76.84999847]
 [76.19999695]
 [76.65000153]
 [75.84999847]
 [75.59999847]
 [75.30000305]
 [75.80000305]
 [75.34999847]
 [75.84999847]
 [75.94999695]
 [75.65000153]
 [75.15000153]
 [75.09999847]
 [75.34999847]
 [75.25      ]
 [75.09999847]
 [75.59999847]
 [76.94999695]
 [76.15000153]
 [76.30000305]
 [76.80000305]
 [76.94999695]
 [78.        ]
 [78.        ]
 [77.90000153]
 [77.69999695]
 [77.34999847]
 [76.09999847]
 [76.25      ]
 [76.80000305]
 [78.30000305]
 [78.34999847]
 [79.94999695]
 [79.55000305]
 [78.55000305]
 [78.84999847]
 [79.34999847]
 [79.30000305]
 [79.55000305]
 [80.        ]
 [79.80000305]
 [80.        ]
 [79.94999695]
 [81.30000305]
 [80.94999695]
 [80.5       ]
 [80.80000

/var/folders/8h/w0jn89bj2w5g85gj4ck7w0t00000gn/T/ipykernel_46622/3393641135.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = np.array(df.drop(['Prediction'],1))


In [5]:
# Split the data into train and test with 90 & 10 % respectively
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train=X_train.reshape(X_train.shape[0],1,1)
X_test=X_test.reshape(X_test.shape[0],1,1)

In [6]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(1, batch_input_shape= (10,X_train.shape[1], X_train.shape[2]),dropout=0.2))
model.add(Dense(1))
model.compile(
    loss="mean_squared_error",
    optimizer='Adam'
)

Metal device set to: Apple M1 Pro


2021-12-26 18:23:52.369793: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-26 18:23:52.369892: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
#Fit model with history to check for overfitting
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40, min_delta=0)
history = model.fit(X_train,y_train,epochs=100,batch_size=10,validation_data=(X_test,y_test),shuffle=False,callbacks=es,verbose=1)

2021-12-26 18:23:52.537596: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100


2021-12-26 18:23:52.951547: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-26 18:23:53.120084: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-26 18:23:53.222437: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [ ]:
y_pred=svr.predict(x_test)

# Unseen Data

In [ ]:
newData = yf.download('0005.hk',"2021-09-25","2021-10-05")

In [ ]:
newData['Prediction']=newData['Close'].shift(-1)
newData.dropna(inplace=True)
x_unseen=newData['Close'].values.reshape(-1,1)
y_unseen=newData['Prediction'].values
print(newData)

In [ ]:
svm_prediction = svr.predict(x_unseen)

In [ ]:
plt.plot(y_unseen, marker='.', label="true")
plt.plot(svm_prediction, 'r',marker='*', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();